In [448]:
import pandas as pd
import numpy as np
import os

In [449]:
data = pd.read_csv("merged_1.csv", index_col=0, parse_dates=True, encoding="latin1")

# data = data.drop(['stn_code','sampling_date','state','location','agency','type','location_monitoring_station'  ], axis=1)
# data = data.drop(['StationId','AQI_Bucket'], axis=1)
data.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2626 entries, 2023-11-28 20:25:00 to 2023-12-07 23:10:00
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   BIKE         0 non-null      float64
 1   CAR          0 non-null      float64
 2   HEAVY        0 non-null      float64
 3   HUMIDITY     73 non-null     float64
 4   PEDESTRIAN   0 non-null      float64
 5   PM10         75 non-null     float64
 6   PM25         75 non-null     float64
 7   TEMPERATURE  73 non-null     float64
 8   V85          0 non-null      float64
dtypes: float64(9)
memory usage: 205.2 KB


In [450]:
data.head()

,BIKE,CAR,HEAVY,HUMIDITY,PEDESTRIAN,PM10,PM25,TEMPERATURE,V85
timestamp,,,,,,,,,
2023-11-28 20:25:00,NaN,NaN,NaN,99.9,NaN,0.6,0.6,-0.3,NaN
2023-11-28 20:30:00,NaN,NaN,NaN,NaN,NaN,0.4,0.4,NaN,NaN
2023-11-28 20:35:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-11-28 20:40:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-11-28 20:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [451]:
data.isnull().sum()

BIKE           2626
CAR            2626
HEAVY          2626
HUMIDITY       2553
PEDESTRIAN     2626
PM10           2551
PM25           2551
TEMPERATURE    2553
V85            2626
dtype: int64

In [452]:
from sklearn.impute import KNNImputer
# from sklearn.impute import SimpleImputer

original_data = data.copy() 

# if there is a column with no values at all, it will be dropped
data = data.dropna(axis=1, how='all')

# for testing use simple imputer, for final use KNNImputer
# imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp = KNNImputer(missing_values=np.nan)
imp.fit(data)


x = imp.transform(data)
data = pd.DataFrame(x, columns=data.columns)








In [453]:
from sklearn.ensemble import IsolationForest


# random_state = np.random.RandomState(42)
model=IsolationForest(n_estimators=10, random_state=42)
model.fit(data)
print(model.get_params())
y_pred_outliers = model.predict(data)

# print the row where the ypred is 1
data['outlier'] = y_pred_outliers
data[data['outlier']==-1]


{'bootstrap': False, 'contamination': 'auto', 'max_features': 1.0, 'max_samples': 'auto', 'n_estimators': 10, 'n_jobs': None, 'random_state': 42, 'verbose': 0, 'warm_start': False}


,HUMIDITY,PM10,PM25,TEMPERATURE,outlier
0,99.9,0.6,0.6,-0.30,-1
1,99.9,0.4,0.4,1.94,-1
2337,99.9,10.7,6.6,1.60,-1
2338,99.9,8.8,5.6,1.60,-1
2453,99.9,16.6,7.0,2.30,-1
...,...,...,...,...,...
2554,99.9,21.8,8.1,2.30,-1
2555,99.9,26.6,7.3,2.30,-1
2623,99.9,13.0,7.4,1.70,-1
2624,99.9,17.1,7.2,1.70,-1


In [454]:
# replace the columns from the original data with the imputed data including the outlier column
original_data['outlier'] = y_pred_outliers
original_data[original_data['outlier']==-1]
original_data = original_data.drop(data.columns, axis=1)
original_data = pd.concat([original_data, data], axis=1)    
original_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5252 entries, 2023-11-28 20:25:00 to 2625
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   BIKE         0 non-null      float64
 1   CAR          0 non-null      float64
 2   HEAVY        0 non-null      float64
 3   PEDESTRIAN   0 non-null      float64
 4   V85          0 non-null      float64
 5   HUMIDITY     2626 non-null   float64
 6   PM10         2626 non-null   float64
 7   PM25         2626 non-null   float64
 8   TEMPERATURE  2626 non-null   float64
 9   outlier      2626 non-null   float64
dtypes: float64(10)
memory usage: 451.3+ KB


In [455]:
original_data.isna().sum()

BIKE           5252
CAR            5252
HEAVY          5252
PEDESTRIAN     5252
V85            5252
HUMIDITY       2626
PM10           2626
PM25           2626
TEMPERATURE    2626
outlier        2626
dtype: int64